<a href="https://colab.research.google.com/github/ErnstHolger/jupyter_notebook/blob/main/OSIsoft_PI_Compression.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [15]:
import math
import datetime
import plotly.express as px

In [3]:
def slope(firstValue, lastValue, threshold):
  diff= (lastValue[0]-firstValue[0])
  return (lastValue[1]+threshold-firstValue[1])/(diff.total_seconds() * 1000)

In [35]:
class compression:
  def __init__(self, threshold):
    self.threshold=threshold
    self.heldPoint=None
    self.upperSlope=0
    self.lowerSlope=0
    self.archivedPoint=None
    self.heldPoint=None

  def calculate(self, timeStamp, value):
    dv=(timeStamp, value)
    if not self.archivedPoint:
      self.archivedPoint = dv
      self.LastRecordedValue = dv
      return dv
    if not self.heldPoint:
      self.heldPoint=dv
      self.upperSlope = slope(self.archivedPoint, self.heldPoint, self.threshold);
      self.lowerSlope = slope(self.archivedPoint, self.heldPoint, -self.threshold);
      return None
    else:
      pointSlope = slope(self.archivedPoint, dv, 0);
      newUpperSlope = slope(self.archivedPoint, dv, self.threshold)
      newLowerSlope = slope(self.archivedPoint, dv, -self.threshold)
      if pointSlope <= self.upperSlope and pointSlope >= self.lowerSlope:
        self.upperSlope = min(self.upperSlope, newUpperSlope);
        self.lowerSlope = min(self.lowerSlope, newLowerSlope);
        self.heldPoint = dv
        return None
      else:
        self.archivedPoint = self.heldPoint
        self.heldPoint = dv
        self.upperSlope = slope(self.archivedPoint, self.heldPoint, self.threshold);
        self.lowerSlope = slope(self.archivedPoint, self.heldPoint, -self.threshold);
        self.lastRecordedValue = dv;
        return self.archivedPoint

In [40]:
x=list(range(100))
y=[math.sin(i/10) for i in x]
dt=datetime.datetime(2022,1,1,8,0,0)
x=[dt + datetime.timedelta(seconds=i) for i in x]

comp=compression(0.025)
xc=[]
yc=[]
for i in range(len(x)):
  pval=comp.calculate(x[i],y[i])
  if pval:
    xc.append(pval[0])
    yc.append(pval[1])

fig=px.line(x=x,y=y)
fig.add_scatter(x=xc,y=yc,name='compressed')
fig.update_layout({'plot_bgcolor': 'rgba(0, 0, 0, 0)',
              'paper_bgcolor': 'rgba(0, 0, 0, 0)',
              'font_family':"Courier New",
              'font_size':14,
              'font_color':"white",
              'title_font_family':"Courier New",
              'title_font_color':"white",})
fig.show()